# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


## Spark SQL

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
sc = pyspark.SparkContext(appName='sga_task')
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-10-09 21:41:19,624 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
!hadoop fs -put clickstream.csv clickstream.csv
clickstream = se.read.option("delimiter", "\t").csv("clickstream.csv", header=True)
clickstream.createOrReplaceTempView("clickstream")

put: `clickstream.csv': File exists


In [4]:
query = """
WITH page_routes AS(
SELECT c.user_id,
       c.session_id,
       c.timestamp,
       COLLECT_LIST(c.event_page) OVER(PARTITION BY c.user_id, c.session_id ORDER BY c.timestamp) AS route,
       ROW_NUMBER() OVER(PARTITION BY c.user_id, c.session_id ORDER BY c.timestamp DESC) AS rn
FROM clickstream c
LEFT JOIN (SELECT cs.user_id,
                  cs.session_id,
                  MIN(cs.timestamp) FILTER(WHERE cs.event_type LIKE '%error%') AS min_tmp
           FROM clickstream cs
           GROUP BY 1,2) min_time ON min_time.user_id = c.user_id
                                  AND min_time.session_id = c.session_id
WHERE (c.timestamp < min_time.min_tmp OR min_time.min_tmp IS NULL) 
      AND c.event_type = 'page' )

SELECT CONCAT_WS('-', route) AS route,
       COUNT(*) cnt_total
FROM page_routes p 
WHERE rn = 1
GROUP BY 1
ORDER BY 2 DESC"""

In [5]:
route = se.sql(query)
route.show(30)

+--------------------+---------+
|               route|cnt_total|
+--------------------+---------+
|                main|     8090|
|        main-archive|     1096|
|         main-rabota|     1039|
|       main-internet|      880|
|          main-bonus|      865|
|           main-news|      760|
|        main-tariffs|      669|
|         main-online|      584|
|          main-vklad|      514|
| main-rabota-archive|      167|
| main-archive-rabota|      167|
|  main-bonus-archive|      139|
|   main-rabota-bonus|      137|
|    main-news-rabota|      134|
|   main-bonus-rabota|      133|
|main-archive-inte...|      131|
|    main-rabota-news|      129|
|main-internet-rabota|      128|
|   main-archive-news|      125|
|main-rabota-internet|      123|
|main-internet-arc...|      123|
|  main-archive-bonus|      117|
|main-tariffs-inte...|      114|
| main-internet-bonus|      114|
|   main-news-archive|      112|
|  main-news-internet|      108|
|main-archive-tariffs|      103|
|  main-in

In [144]:
route_pd = route.toPandas()

In [146]:
route_pd.to_csv('spark_sql.csv')

## Spark DF

In [6]:
from pyspark.sql.window import Window
from pyspark.sql import functions as func

In [7]:
errors = clickstream.alias('e')\
                    .filter(func.col('e.event_type').like('%error%'))\
                    .groupBy('e.user_id', 'e.session_id').agg(func.min('e.timestamp').alias('min_tmp'))
cleaned_session = clickstream.alias('c')\
                             .join(errors.alias('cf'), on = ['user_id', 'session_id'], how = 'left')\
                             .filter(((func.col('c.timestamp') < func.col('cf.min_tmp')) | func.col('cf.min_tmp').isNull()) & (func.col('c.event_type') == 'page'))\
                             .withColumn('rn', func.row_number().over(Window.partitionBy('c.user_id', 'c.session_id').orderBy(func.col('c.timestamp').desc())))
cleaned_routes  = cleaned_session.alias('cl')\
                                 .withColumn('route', func.collect_list('cl.event_page').over(Window.partitionBy('cl.user_id', 'cl.session_id').orderBy('cl.timestamp')))\
                                 .filter(func.col('cl.rn') == 1)
all_routes = cleaned_routes.groupBy(func.concat_ws('-','route').alias('route'))\
                           .agg(func.count('*').alias('cnt_total'))\
                           .orderBy('cnt_total', ascending = False)
all_routes.show(30)

+--------------------+---------+
|               route|cnt_total|
+--------------------+---------+
|                main|     8090|
|        main-archive|     1096|
|         main-rabota|     1039|
|       main-internet|      880|
|          main-bonus|      865|
|           main-news|      760|
|        main-tariffs|      669|
|         main-online|      584|
|          main-vklad|      514|
| main-rabota-archive|      167|
| main-archive-rabota|      167|
|  main-bonus-archive|      139|
|   main-rabota-bonus|      137|
|    main-news-rabota|      134|
|   main-bonus-rabota|      133|
|main-archive-inte...|      131|
|    main-rabota-news|      129|
|main-internet-rabota|      128|
|   main-archive-news|      125|
|main-rabota-internet|      123|
|main-internet-arc...|      123|
|  main-archive-bonus|      117|
|main-tariffs-inte...|      114|
| main-internet-bonus|      114|
|   main-news-archive|      112|
|  main-news-internet|      108|
|main-archive-tariffs|      103|
|  main-in

In [149]:
all_routes_pd = all_routes.toPandas()

In [150]:
all_routes_pd.to_csv('spark_df.csv')

## Spark RDD

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
rdd = clickstream.rdd

In [14]:
errors = (rdd.filter(lambda r: 'error' in r.event_type)
             .map(lambda r: ((r.user_id, r.session_id), r.timestamp)).combineByKey(lambda x: x, min, min))

clean_route = (rdd
               .filter(lambda r: r.event_type == 'page')
               .map(lambda r: ((r.user_id, r.session_id), r))
               .leftOuterJoin(errors)
               .filter(lambda x: x[1][1] is None or x[1][0].timestamp < x[1][1])
               .map(lambda x: x[1][0])
               .map(lambda r: ((r.user_id, r.session_id), [r.event_page]))
               .reduceByKey(lambda a, b: a + b)
               .map(lambda x: ("-".join(x[1]), 1))
               .reduceByKey(lambda a, b: a + b)
               .sortBy(lambda x: x[1], ascending=False).take(30))

clean_route


[('main', 8090),
 ('main-archive', 1093),
 ('main-rabota', 1036),
 ('main-internet', 880),
 ('main-bonus', 865),
 ('main-news', 760),
 ('main-tariffs', 668),
 ('main-online', 584),
 ('main-vklad', 514),
 ('main-archive-rabota', 167),
 ('main-rabota-archive', 167),
 ('main-bonus-archive', 139),
 ('main-rabota-bonus', 137),
 ('main-news-rabota', 134),
 ('main-bonus-rabota', 133),
 ('main-archive-internet', 131),
 ('main-rabota-news', 129),
 ('main-internet-rabota', 128),
 ('main-archive-news', 125),
 ('main-internet-archive', 123),
 ('main-rabota-internet', 123),
 ('main-archive-bonus', 117),
 ('main-tariffs-internet', 114),
 ('main-internet-bonus', 113),
 ('main-news-archive', 112),
 ('main-news-internet', 108),
 ('main-archive-tariffs', 103),
 ('main-internet-news', 102),
 ('main-tariffs-archive', 101),
 ('main-main', 94)]

In [159]:
clean_route_df = pd.DataFrame(clean_route)
clean_route_df.to_csv('rdd.csv')